In [1]:
import cobra
import numpy as np
import pandas as pd
import sys

source_dir = '../src'
sys.path.append(source_dir)

from get_gene_transcript_dictionary import get_gene_transcript_dictionary
from get_reaction_transcript_dictionary import get_reaction_transcript_dictionary
from add_column_to_13c_flux_df import add_column_to_13c_flux_df


### Load genome scale model

In [2]:
model = cobra.io.json.load_json_model("../genome_scale_models/iYLI647_corr_2.json")
model

Name,model
Memory address,1108cb370
Number of metabolites,1121
Number of reactions,1348
Number of genes,648
Number of groups,0
Objective expression,1.0*biomass_C - 1.0*biomass_C_reverse_c1d5c
Compartments,"c, e, m, n, x, r, g, v"


### Load transcriptomics data

In [3]:
# load this file data/transcriptomics/fpkm_counts.csv
transcriptomics_df = pd.read_csv("../data/transcriptomics/fpkm_counts.csv")
transcriptomics_df

,JGI_id,YALI0_id,glucose_1,glucose_2,glucose_3,glycerol_1,glycerol_2,glycerol_3,oleic_acid_1,oleic_acid_2,oleic_acid_3
0,jgi.p|Yarli1|64471,YALI0A00110g,4.88,2.64,3.23,1.21,1.11,3.61,6.77,13.32,7.67
1,jgi.p|Yarli1|64472,YALI0A00132g,1896.43,1845.77,1682.56,1234.99,1189.24,1120.35,864.64,1056.63,928.51
2,jgi.p|Yarli1|64473,YALI0A00154g,31.34,29.21,21.92,215.23,137.85,182.81,1867.59,2153.21,1958.68
3,jgi.p|Yarli1|64474,YALI0A00176g,3.67,3.85,3.15,3.07,1.74,2.09,4.13,2.43,4.34
4,jgi.p|Yarli1|64475,YALI0A00198g,0.00,0.17,0.00,0.00,0.50,1.29,1.55,0.81,1.75
...,...,...,...,...,...,...,...,...,...,...,...
6442,jgi.p|Yarli1|70913,YALI0F32043g,20.63,17.60,17.12,13.94,13.84,11.94,10.76,12.38,10.37
6443,jgi.p|Yarli1|70914,YALI0F32065g,192.58,151.54,270.76,206.89,262.66,280.82,310.55,384.22,313.24
6444,jgi.p|Yarli1|70915,YALI0F32131g,0.12,0.66,0.14,0.34,0.29,0.64,1.01,0.58,1.33
6445,jgi.p|Yarli1|70916,YALI0F32153g,26.16,27.73,28.77,39.22,36.95,39.18,26.98,22.05,23.58


### Make gene -> transcript dictionaries for each condition

In [11]:
glucose_1_gene_transcripts = get_gene_transcript_dictionary(transcriptomics_df, 'glucose_1')
glucose_2_gene_transcripts = get_gene_transcript_dictionary(transcriptomics_df, 'glucose_2')
glucose_3_gene_transcripts = get_gene_transcript_dictionary(transcriptomics_df, 'glucose_3')

glucose_2_gene_transcripts


{'': inf,
 'YALI0A00110g': 2.64,
 'YALI0A00132g': 1845.77,
 'YALI0A00154g': 29.21,
 'YALI0A00176g': 3.85,
 'YALI0A00198g': 0.17,
 'YALI0A00212g': 9.49,
 'YALI0A00264g': 496.98,
 'YALI0A00286g': 272.97,
 'YALI0A00330g': 27.91,
 'YALI0A00352g': 1682.06,
 'YALI0A00369g': 0.97,
 'YALI0A00374g': 1.49,
 'YALI0A00396g': 37.67,
 'YALI0A00418g': 4.19,
 'YALI0A00440g': 147.92,
 'YALI0A00506g': 46.09,
 'YALI0A00528g': 0.16,
 'YALI0A00550g': 4.65,
 'YALI0A00572g': 0.33,
 'YALI0A00594g': 539.01,
 'YALI0A00616g': 35.88,
 'YALI0A00638g': 55.59,
 'YALI0A00660g': 20.1,
 'YALI0A00682g': 0.38,
 'YALI0A00759g': 30.54,
 'YALI0A00781g': 53.41,
 'YALI0A00803g': 156.08,
 'YALI0A00825g': 13.7,
 'YALI0A00847g': 27.33,
 'YALI0A00869g': 0.0,
 'YALI0A00891g': 146.87,
 'YALI0A00913g': 57.43,
 'YALI0A00935g': 2.79,
 'YALI0A00957g': 18.64,
 'YALI0A00979g': 29.67,
 'YALI0A01001g': 39.69,
 'YALI0A01023g': 99.56,
 'YALI0A01045g': 74.28,
 'YALI0A01067g': 195.78,
 'YALI0A01089g': 34.99,
 'YALI0A01111g': 34.65,
 'YALI0A011

### Make reaction -> transcript dictionaries for each condition

In [5]:
glucose_1_reaction_transcripts = get_reaction_transcript_dictionary(model, glucose_1_gene_transcripts)
glucose_2_reaction_transcripts = get_reaction_transcript_dictionary(model, glucose_2_gene_transcripts)
glucose_3_reaction_transcripts = get_reaction_transcript_dictionary(model, glucose_3_gene_transcripts)

glucose_1_reaction_transcripts

{'13BGH': 35.55,
 '13BGHe': 35.55,
 'ASADi': 336.06,
 '2DDA7Ptm': inf,
 '2DHPtm': inf,
 '2OHPH_5tn': inf,
 '2OMPH_5tm': inf,
 '2OMPH_5tn': inf,
 'ASPKi': 113.22,
 '34HPPOR': inf,
 '34HPPt2m': inf,
 '34HPPt2p': inf,
 '13GS': 0.82,
 '3C4MOPtm': inf,
 '3DSPHR': 113.14,
 '3HAO': 90.83,
 'GLCS2': 204.07,
 'SBTD_D2': 61.35,
 '3OPHB_5tm': inf,
 '4ABUTNtm': inf,
 '4ABUTtm': inf,
 '4ABZtm': inf,
 '4H2OGLTtm': inf,
 '4H2OGLTtp': inf,
 '4HBZtm': inf,
 '4HPRO_LTtm': inf,
 '4HTHRS': 182.8,
 '5AOPt2': 0.2,
 '5AOPtm': inf,
 '5MTHFtm': inf,
 '6PGLter': inf,
 '8AONNt2': inf,
 'AABTN': inf,
 'AASAD1': 167.01,
 'TRE6PP': 38.15,
 'ABTA': 51.69,
 'ABTt': inf,
 'ABUTDm': 0.22,
 'ABUTt2': 19.65,
 'TRE6PS': 18.59,
 'ACACT1m': 82.98,
 'ACACT4p': 447.71,
 'ACACT5p': 447.71,
 'ACACT6p': 447.71,
 'ACACT7p': 447.71,
 'ACACT8p': 447.71,
 'ACACT9p': 447.71,
 'PC': 219.07,
 'ACGAM6PS': 425.44,
 'ACGAMPM': 53.15,
 'ACGKm': 177.78,
 'ACGSm': 26.85,
 'ACOTAim': 179.71,
 'AGPRim': 177.78,
 'ACOAH': 173.51,
 'ACOAO4p': 82

### Run E-Flux2 for Glucose 1

In [6]:
with model:
    model.medium = {
      'EX_glc(e)': np.inf,
      'EX_h2o(e)': 1000.0,
      'EX_h(e)': 1000.0,
      'EX_nh4(e)': 1000.0,
      'EX_o2(e)': 1000.0,
      'EX_pi(e)': 1000.0,
      'EX_so4(e)': 1000.0,
    }

    # set the flux bounds for each reaction using the transcriptomics data    
    for r in model.reactions:
        if 'EX_' not in r.id:
            # if r.gene_reaction_rule:
            if r.lower_bound < 0.0:
                r.lower_bound = -glucose_1_reaction_transcripts[r.id]

            if r.upper_bound > 0.0:
                r.upper_bound = glucose_1_reaction_transcripts[r.id]
        else:
            print(f'{r.id} is not transcript constrained')

    # find the optimal solution
    glucose_1_eflux2_solution = cobra.flux_analysis.pfba(model)

    # make a list of dictionaries with the reaction id, name, flux, and absolute flux
    reactions = []
    for reaction_id, flux in glucose_1_eflux2_solution.fluxes.items():
      reactions.append({
        'reaction_id': reaction_id,
        'reaction_name': model.reactions.get_by_id(reaction_id).name,
        'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
        'flux': flux,
        'absolute_flux': abs(flux), # use for sorting, then drop
      })

    # make a dataframe from the list of dictionaries
    glucose_1_gsm_eflux2_df = pd.DataFrame(reactions)

    # sort the dataframe by absolute flux
    glucose_1_gsm_eflux2_df = glucose_1_gsm_eflux2_df.sort_values(by=['absolute_flux'], ascending=False)

    # drop the absolute flux column
    glucose_1_gsm_eflux2_df = glucose_1_gsm_eflux2_df.drop(columns=['absolute_flux'])

    # get the carbon uptake flux value
    uptake_row = glucose_1_gsm_eflux2_df[glucose_1_gsm_eflux2_df['reaction_id'] == 'EX_glc(e)']
    uptake_flux = -1 * uptake_row['flux'].iloc[0]

    # scale all fluxes, so uptake flux is 100
    glucose_1_gsm_eflux2_df['flux'] = glucose_1_gsm_eflux2_df['flux'] * (100 / uptake_flux)

    # save the dataframe to a csv file
    glucose_1_gsm_eflux2_df.to_csv('../results/eflux2_gsm_fluxes/glucose_1_gsm_eflux2.csv', index=False)

    display(glucose_1_gsm_eflux2_df.head(30))


EX_13BDglcn(e) is not transcript constrained
EX_4abut(e) is not transcript constrained
EX_5aop(e) is not transcript constrained
EX_8aonn(e) is not transcript constrained
EX_abt(e) is not transcript constrained
EX_ac(e) is not transcript constrained
EX_ade(e) is not transcript constrained
EX_adn(e) is not transcript constrained
EX_akg(e) is not transcript constrained
EX_ala_L(e) is not transcript constrained
EX_alltn(e) is not transcript constrained
EX_alltt(e) is not transcript constrained
EX_amet(e) is not transcript constrained
EX_arab_D(e) is not transcript constrained
EX_arab_L(e) is not transcript constrained
EX_arg_L(e) is not transcript constrained
EX_asn_L(e) is not transcript constrained
EX_asp_L(e) is not transcript constrained
EX_buca(e) is not transcript constrained
EX_btn(e) is not transcript constrained
EX_chol(e) is not transcript constrained
EX_cit(e) is not transcript constrained
EX_co2(e) is not transcript constrained
EX_crn(e) is not transcript constrained
EX_csn(e) 

,reaction_id,reaction_name,full_reaction,flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-666.959559
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,366.566172
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],365.261313
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],365.261313
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-320.522192
421,EX_h2o(e),H2O exchange,h2o[e] <=>,320.522192
617,CYOR_u6m,ubiquinol 6 cytochrome c reductase,2.0 ficytc[m] + 1.5 h[m] + q6h2[m] --> 2.0 foc...,316.638002
639,NADH2_u6cm,NADH dehydrogenase cytosolicmitochondrial,h[c] + nadh[c] + q6[m] --> nad[c] + q6h2[m],281.289436
291,GLUDy,glutamate dehydrogenase NADP,glu_L[c] + h2o[c] + nadp[c] <=> akg[c] + h[c] ...,-187.019702
391,EX_co2(e),CO2 exchange,co2[e] -->,184.301252


### Run E-Flux2 for Glucose 2

In [7]:
with model:
    model.medium = {
      'EX_glc(e)': np.inf,
      'EX_h2o(e)': 1000.0,
      'EX_h(e)': 1000.0,
      'EX_nh4(e)': 1000.0,
      'EX_o2(e)': 1000.0,
      'EX_pi(e)': 1000.0,
      'EX_so4(e)': 1000.0,
    }

    # set the flux bounds for each reaction using the transcriptomics data    
    for r in model.reactions:
        if 'EX_' not in r.id:
            # if r.gene_reaction_rule:
            if r.lower_bound < 0.0:
                r.lower_bound = -glucose_2_reaction_transcripts[r.id]

            if r.upper_bound > 0.0:
                r.upper_bound = glucose_2_reaction_transcripts[r.id]
        else:
            print(f'{r.id} is not transcript constrained')

    # find the optimal solution
    glucose_2_eflux2_solution = cobra.flux_analysis.pfba(model)

    # make a list of dictionaries with the reaction id, name, flux, and absolute flux
    reactions = []
    for reaction_id, flux in glucose_2_eflux2_solution.fluxes.items():
      reactions.append({
        'reaction_id': reaction_id,
        'reaction_name': model.reactions.get_by_id(reaction_id).name,
        'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
        'flux': flux,
        'absolute_flux': abs(flux), # use for sorting, then drop
      })

    # make a dataframe from the list of dictionaries
    glucose_2_gsm_eflux2_df = pd.DataFrame(reactions)

    # sort the dataframe by absolute flux
    glucose_2_gsm_eflux2_df = glucose_2_gsm_eflux2_df.sort_values(by=['absolute_flux'], ascending=False)

    # drop the absolute flux column
    glucose_2_gsm_eflux2_df = glucose_2_gsm_eflux2_df.drop(columns=['absolute_flux'])

    # get the carbon uptake flux value
    uptake_row = glucose_2_gsm_eflux2_df[glucose_2_gsm_eflux2_df['reaction_id'] == 'EX_glc(e)']
    uptake_flux = -1 * uptake_row['flux'].iloc[0]

    # scale all fluxes, so uptake flux is 100
    glucose_2_gsm_eflux2_df['flux'] = glucose_2_gsm_eflux2_df['flux'] * (100 / uptake_flux)

    # save the dataframe to a csv file
    glucose_2_gsm_eflux2_df.to_csv('../results/eflux2_gsm_fluxes/glucose_2_gsm_eflux2.csv', index=False)

    display(glucose_2_gsm_eflux2_df.head(30))


EX_13BDglcn(e) is not transcript constrained
EX_4abut(e) is not transcript constrained
EX_5aop(e) is not transcript constrained
EX_8aonn(e) is not transcript constrained
EX_abt(e) is not transcript constrained
EX_ac(e) is not transcript constrained
EX_ade(e) is not transcript constrained
EX_adn(e) is not transcript constrained
EX_akg(e) is not transcript constrained
EX_ala_L(e) is not transcript constrained
EX_alltn(e) is not transcript constrained
EX_alltt(e) is not transcript constrained
EX_amet(e) is not transcript constrained
EX_arab_D(e) is not transcript constrained
EX_arab_L(e) is not transcript constrained
EX_arg_L(e) is not transcript constrained
EX_asn_L(e) is not transcript constrained
EX_asp_L(e) is not transcript constrained
EX_buca(e) is not transcript constrained
EX_btn(e) is not transcript constrained
EX_chol(e) is not transcript constrained
EX_cit(e) is not transcript constrained
EX_co2(e) is not transcript constrained
EX_crn(e) is not transcript constrained
EX_csn(e) 

,reaction_id,reaction_name,full_reaction,flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-667.557678
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,366.662025
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],365.378556
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],365.378556
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-316.963941
421,EX_h2o(e),H2O exchange,h2o[e] <=>,316.963941
617,CYOR_u6m,ubiquinol 6 cytochrome c reductase,2.0 ficytc[m] + 1.5 h[m] + q6h2[m] --> 2.0 foc...,316.873988
639,NADH2_u6cm,NADH dehydrogenase cytosolicmitochondrial,h[c] + nadh[c] + q6[m] --> nad[c] + q6h2[m],282.104852
957,CO2t,CO2 transporter via diffusion,co2[e] <=> co2[c],-185.010774
391,EX_co2(e),CO2 exchange,co2[e] -->,185.010774


### Run E-Flux2 for Glucose 3

In [8]:
with model:
    model.medium = {
      'EX_glc(e)': np.inf,
      'EX_h2o(e)': 1000.0,
      'EX_h(e)': 1000.0,
      'EX_nh4(e)': 1000.0,
      'EX_o2(e)': 1000.0,
      'EX_pi(e)': 1000.0,
      'EX_so4(e)': 1000.0,
    }

    # set the flux bounds for each reaction using the transcriptomics data    
    for r in model.reactions:
        if 'EX_' not in r.id:
            # if r.gene_reaction_rule:
            if r.lower_bound < 0.0:
                r.lower_bound = -glucose_3_reaction_transcripts[r.id]

            if r.upper_bound > 0.0:
                r.upper_bound = glucose_3_reaction_transcripts[r.id]
        else:
            print(f'{r.id} is not transcript constrained')

    # find the optimal solution
    glucose_3_eflux2_solution = cobra.flux_analysis.pfba(model)

    # make a list of dictionaries with the reaction id, name, flux, and absolute flux
    reactions = []
    for reaction_id, flux in glucose_3_eflux2_solution.fluxes.items():
      reactions.append({
        'reaction_id': reaction_id,
        'reaction_name': model.reactions.get_by_id(reaction_id).name,
        'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
        'flux': flux,
        'absolute_flux': abs(flux), # use for sorting, then drop
      })

    # make a dataframe from the list of dictionaries
    glucose_3_gsm_eflux2_df = pd.DataFrame(reactions)

    # sort the dataframe by absolute flux
    glucose_3_gsm_eflux2_df = glucose_3_gsm_eflux2_df.sort_values(by=['absolute_flux'], ascending=False)

    # drop the absolute flux column
    glucose_3_gsm_eflux2_df = glucose_3_gsm_eflux2_df.drop(columns=['absolute_flux'])

    # get the carbon uptake flux value
    uptake_row = glucose_3_gsm_eflux2_df[glucose_3_gsm_eflux2_df['reaction_id'] == 'EX_glc(e)']
    uptake_flux = -1 * uptake_row['flux'].iloc[0]

    # scale all fluxes, so uptake flux is 100
    glucose_3_gsm_eflux2_df['flux'] = glucose_3_gsm_eflux2_df['flux'] * (100 / uptake_flux)

    # save the dataframe to a csv file
    glucose_3_gsm_eflux2_df.to_csv('../results/eflux2_gsm_fluxes/glucose_3_gsm_eflux2.csv', index=False)

    display(glucose_3_gsm_eflux2_df.head(30))


EX_13BDglcn(e) is not transcript constrained
EX_4abut(e) is not transcript constrained
EX_5aop(e) is not transcript constrained
EX_8aonn(e) is not transcript constrained
EX_abt(e) is not transcript constrained
EX_ac(e) is not transcript constrained
EX_ade(e) is not transcript constrained
EX_adn(e) is not transcript constrained
EX_akg(e) is not transcript constrained
EX_ala_L(e) is not transcript constrained
EX_alltn(e) is not transcript constrained
EX_alltt(e) is not transcript constrained
EX_amet(e) is not transcript constrained
EX_arab_D(e) is not transcript constrained
EX_arab_L(e) is not transcript constrained
EX_arg_L(e) is not transcript constrained
EX_asn_L(e) is not transcript constrained
EX_asp_L(e) is not transcript constrained
EX_buca(e) is not transcript constrained
EX_btn(e) is not transcript constrained
EX_chol(e) is not transcript constrained
EX_cit(e) is not transcript constrained
EX_co2(e) is not transcript constrained
EX_crn(e) is not transcript constrained
EX_csn(e) 

,reaction_id,reaction_name,full_reaction,flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-667.221169
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,366.608097
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],365.312594
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],365.312594
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-318.965861
421,EX_h2o(e),H2O exchange,h2o[e] <=>,318.965861
617,CYOR_u6m,ubiquinol 6 cytochrome c reductase,2.0 ficytc[m] + 1.5 h[m] + q6h2[m] --> 2.0 foc...,316.741219
639,NADH2_u6cm,NADH dehydrogenase cytosolicmitochondrial,h[c] + nadh[c] + q6[m] --> nad[c] + q6h2[m],281.646088
391,EX_co2(e),CO2 exchange,co2[e] -->,184.611587
957,CO2t,CO2 transporter via diffusion,co2[e] <=> co2[c],-184.611587


### Glucose Central Metabolism E-Flux2

In [10]:
# load glucose 13C MFA data
glucose_13c_mfa_df = pd.read_csv('../data/13c_mfa/glucose_13c_mfa.csv')

# only keep the reactions that are mapped to the GSM
glucose_13c_mfa_df.dropna(subset = ["reaction_ids"], inplace=True)

# add the GSM flux predictions to the 13C-MFA dataframe
glucose_central_eflux2_df = add_column_to_13c_flux_df(glucose_13c_mfa_df, glucose_1_gsm_eflux2_df, 'Glucose 1 Flux')
glucose_central_eflux2_df = add_column_to_13c_flux_df(glucose_central_eflux2_df, glucose_2_gsm_eflux2_df, 'Glucose 2 Flux')
glucose_central_eflux2_df = add_column_to_13c_flux_df(glucose_central_eflux2_df, glucose_3_gsm_eflux2_df, 'Glucose 3 Flux')

# save the central fluxes as .csv
glucose_central_eflux2_df.to_csv('../results/eflux2_central_fluxes/glucose_central_eflux2.csv', index=False)

glucose_central_eflux2_df

,pathway,compartment,13C_mfa_reaction,reaction_ids,13C_mfa_flux,13C_mfa_flux_95%_lower_bound,13C_mfa_flux_95%_upper_bound,Forward Flux,Backward Flux,Stdev,Glucose 1 Flux,Glucose 2 Flux,Glucose 3 Flux
0,substrate_uptake,cytosol,Glucose(substrate) + ATP == G6P,GLCt1,100.0000,100.0000,100.0000,[],[],0.0000,1.000000e+02,100.000000,100.000000
1,substrate_uptake,cytosol,Glycerol(substrate) == GLYC,reverse_GLYCt,0.0000,0.0000,1.2414,[],[],0.3167,-7.248396e+00,-9.164436,-8.086446
2,substrate_uptake,cytosol,Acetate(substrate) == AC,ACt2r,0.0678,0.0000,0.7662,[],[],0.1955,0.000000e+00,0.000000,0.000000
3,substrate_uptake,cytosol,oleic_acid(substrate) == AC,OCDCEAt,0.0000,0.0000,0.0000,[],[],0.0000,0.000000e+00,0.000000,0.000000
4,emp,cytosol,G6P == F6P,PGI,72.6860,58.4274,78.3064,72.686,0,5.0712,-4.354580e+01,-44.923378,-44.148332
5,emp,cytosol,F6P + ATP == FBP,PFK or reverse_FBP,80.4599,71.9969,81.0889,149.0655,68.6056,2.3194,0.000000e+00,0.000000,0.000000
6,emp,cytosol,FBP == DHAP + GAP,FBA,80.4599,71.9969,81.0889,162.3111,81.8512,2.3194,0.000000e+00,0.000000,0.000000
7,emp,cytosol,DHAP == GAP,TPI,-19.5394,-25.9334,-18.2523,7.6658,27.2052,1.9595,2.307489e+01,21.057458,22.192491
8,emp,cytosol,GAP == G3P + ATP + NADH,GAPD,66.2012,54.6225,66.7364,380.1085,313.9073,3.0903,9.697373e+01,95.384146,96.278467
9,emp,cytosol,G3P == PEP,ENO,65.6276,53.6544,66.0194,1469.4,1403.8,3.1543,8.852677e+01,87.075647,87.892067
